In [1]:
# Leer archivo bosch_aic_datathon_limpio.csv
import pandas as pd

# Leer archivo bosch_aic_datathon_limpio.csv
data = pd.read_csv("bosch_aic_datathon_limpio.csv")


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Preparando los datos
X = data.drop(['MOTIVO DE LA RENUNCIA'], axis=1)
y = data['MOTIVO DE LA RENUNCIA']

# Seleccionar solo columnas numéricas y categóricas
X = X.select_dtypes(include=['int64', 'float64', 'category'])

# Rellenar valores faltantes con la mediana para columnas numéricas solo si son numéricas
for column in X.select_dtypes(include=['int64', 'float64']).columns:
    X[column] = X[column].fillna(X[column].median())

# Codificar la variable objetivo
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Identificar y eliminar clases con pocos ejemplos
class_counts = np.bincount(y_encoded)
classes_to_keep = np.where(class_counts > 10)[0]
mask = np.isin(y_encoded, classes_to_keep)
X_filtered = X[mask]
y_filtered = y_encoded[mask]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, stratify=y_filtered)

# Crear un pipeline para transformar los datos
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, X_train.select_dtypes(include=['int64', 'float64']).columns),
        ('cat', categorical_transformer, X_train.select_dtypes(include=['category']).columns)
    ]
)

# Crear un modelo de regresión logística multiclase
clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Crear un pipeline que primero transforme los datos y luego entrene el modelo
model = make_pipeline(preprocessor, clf)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir los valores de la variable objetivo para los datos de prueba
y_pred = model.predict(X_test)

# Mostrar el reporte de clasificación
print(classification_report(y_test, y_pred, target_names=le.classes_[classes_to_keep]))

                            precision    recall  f1-score   support

       CAMBIO DE DOMICILIO       0.00      0.00      0.00        18
                  ESTUDIOS       0.00      0.00      0.00        14
   INSATISFACCIÓN PERSONAL       0.00      0.00      0.00        19
 NO REGRESO   NO DIO RAZÓN       0.33      0.72      0.46        43
               OTRO EMPLEO       0.31      0.71      0.43        62
    PROB PERSONALES Y FAM        0.19      0.10      0.13        39
           RAZONES MÉDICAS       0.00      0.00      0.00         6
 TRA AUSENTISMOS CONTINUOS       0.00      0.00      0.00        14
            TRA JUBILACIÓN       0.78      1.00      0.88         7
  TRA LFT Y REGLAMENTO INT       0.00      0.00      0.00         2
 TRA LFT Y REGLAMENTO INT        0.00      0.00      0.00         2
TRANSFERENCIA ENTRE PLANTA       0.50      0.09      0.15        47

                  accuracy                           0.33       273
                 macro avg       0.18      0.2

/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codedspirit/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av